# Project Milestone Two: Modeling and Feature Engineering

### Due: Midnight on April 13 (with 2-hour grace period) and worth 25 points

### Overview

This milestone builds on your work from Milestone 1. You will:

1. Evaluate baseline models using default settings.
2. Engineer new features and re-evaluate models.
3. Use feature selection techniques to find promising subsets.
4. Select the top 3 models and fine-tune them for optimal performance.

You must do all work in this notebook and upload to your team leader's account in Gradescope. There is no
Individual Assessment for this Milestone. 


In [1]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

import warnings
from sklearn.exceptions import ConvergenceWarning


# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor

# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 42

# =============================
# Utility Functions
# =============================

# Format y-axis labels as dollars with commas (optional)
def dollar_format(x, pos):
    return f'${x:,.0f}'

# Convert seconds to HH:MM:SS format
def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))



## Prelude: Load your Preprocessed Dataset from Milestone 1

In Milestone 1, you handled missing values, encoded categorical features, and explored your data. Before you begin this milestone, you’ll need to load that cleaned dataset and prepare it for modeling.

Here’s what to do:

1. Return to your Milestone 1 notebook and rerun your code through Part 3, where your dataset was fully cleaned (assume it’s called `df_cleaned`).

2. **Save** the cleaned dataset to a file by running:

>   df_cleaned.to_csv("zillow_cleaned.csv", index=False)

3. Switch to this notebook and **load** the saved data:

>   df = pd.read_csv("zillow_cleaned.csv")

4. Create a **train/test split** using `train_test_split`.  
   
6. **Standardize** the features (but not the target!) using **only the training data.** This ensures consistency across models without introducing data leakage from the test set:

>   scaler = StandardScaler()   
>   X_train_scaled = scaler.fit_transform(X_train)    
  
**Notes:** 

- You will not use the testing set during this milestone — it’s reserved for final evaluation later.
- You will have to redo the scaling step when you introduce new features (which have to be scaled as well).


In [2]:
# Add as many code cells as you need
df_clean = pd.read_csv('zillow_cleaned.csv')

In [3]:
df_clean.head()


,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,garagecarcnt,...,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,unitcnt,yearbuilt,numberofstories,censustractandblock,taxvaluedollarcnt
0,1.0,3.5,4.0,6.0,3.5,3100.0,3100.0,6059.0,3.0,2.0,...,53571.0,1286.0,118849.0,96978.0,0.0,1.0,1998.0,1.0,6.059063e+13,1023282.0
1,1.0,1.0,2.0,6.0,1.0,1465.0,1465.0,6111.0,1.0,1.0,...,13091.0,2061.0,118849.0,97099.0,5.0,1.0,1967.0,1.0,6.111001e+13,464000.0
2,1.0,2.0,3.0,6.0,2.0,1243.0,1243.0,6059.0,2.0,2.0,...,21412.0,1286.0,118849.0,97078.0,6.0,1.0,1962.0,1.0,6.059022e+13,564778.0
3,1.0,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,3.0,2.0,...,396551.0,3101.0,118849.0,96330.0,0.0,1.0,1970.0,1.0,6.037300e+13,145143.0
4,1.0,3.0,3.0,8.0,3.0,1312.0,1312.0,6037.0,3.0,2.0,...,12447.0,3101.0,268548.0,96451.0,0.0,1.0,1964.0,1.0,6.037124e+13,119407.0


In [4]:
#split the data into train and test sets
X = df_clean.drop(columns=['taxvaluedollarcnt'])
y = df_clean['taxvaluedollarcnt']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

### Part 1: Baseline Modeling [3 pts]

Apply the following regression models to the scaled training dataset using **default parameters**:

- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree Regression
- Bagging
- Random Forest
- Gradient Boosting Trees

For each model:
- Use **repeated cross-validation** (e.g., 5 folds, 5 repeats).
- Report the **mean and standard deviation of CV RMSE Score** across all folds in a table. 


In [5]:
#Linear regression using repeated cross-validation 5 folds and 5 repeats
lr = LinearRegression()
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=random_state)  
scores_lr = cross_val_score(lr, X_train_scaled, y_train, cv=cv, scoring='neg_mean_squared_error')

rmse_scores_lr = np.sqrt(-scores_lr)
rmse_mean_lr = rmse_scores_lr.mean()
rmse_std_lr = rmse_scores_lr.std()
rmse_table = pd.DataFrame({'RMSE Mean': [rmse_mean_lr], 'RMSE Std': [rmse_std_lr]})
rmse_table.index = ['Linear Regression']
print(rmse_table)

                       RMSE Mean      RMSE Std
Linear Regression  296756.896751  44392.551961


In [6]:
#Ridge regression using repeated cross-validation 5 folds and 5 repeats
ridge = Ridge(random_state=random_state)
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=random_state)
scores_rr = cross_val_score(ridge, X_train_scaled, y_train, cv=cv, scoring='neg_mean_squared_error')

rmse_scores_rr = np.sqrt(-scores_rr )
rmse_mean_rr  = rmse_scores_rr .mean()
rmse_std_rr  = rmse_scores_rr .std()
rmse_table.loc['Ridge'] = [rmse_mean_rr , rmse_std_rr ]
print(rmse_table)

                       RMSE Mean      RMSE Std
Linear Regression  296756.896751  44392.551961
Ridge              289219.047275  19222.851917


In [ ]:

#Lasso regression using repeated cross-validation 5 folds and 5 repeats
# Suppress ConvergenceWarnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    lasso = Lasso(random_state=random_state)
    cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=random_state)
    scores_l = cross_val_score(lasso, X_train_scaled, y_train, cv=cv, scoring='neg_mean_squared_error')

rmse_scores_l = np.sqrt(-scores_l)
rmse_mean_l = rmse_scores_l.mean()
rmse_std_l = rmse_scores_l.std()
rmse_table.loc['Lasso'] = [rmse_mean_l, rmse_std_l]
print(rmse_table)

                       RMSE Mean      RMSE Std
Linear Regression  296756.896751  44392.551961
Ridge              289219.047275  19222.851917
Lasso              293213.692049  33334.707386


In [8]:
#ElasticNet regression using repeated cross-validation 5 folds and 5 repeats
elastic_net = ElasticNet(random_state=random_state)
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=random_state)
scores_en = cross_val_score(elastic_net, X_train_scaled, y_train, cv=cv, scoring='neg_mean_squared_error')

rmse_scores_en = np.sqrt(-scores_en)
rmse_mean_en = rmse_scores_en.mean()
rmse_std_en = rmse_scores_en.std()
rmse_table.loc['ElasticNet'] = [rmse_mean_en, rmse_std_en]
print(rmse_table)

                       RMSE Mean      RMSE Std
Linear Regression  296756.896751  44392.551961
Ridge              289219.047275  19222.851917
Lasso              293213.692049  33334.707386
ElasticNet         294075.484151   4496.946761


In [9]:
#Decision Tree Regressor using repeated cross-validation 5 folds and 5 repeats
dt = DecisionTreeRegressor(random_state=random_state)
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=random_state)
scores_dt = cross_val_score(dt, X_train_scaled, y_train, cv=cv, scoring='neg_mean_squared_error')

rmse_scores_dt = np.sqrt(-scores_dt)
rmse_mean_dt = rmse_scores_dt.mean()
rmse_std_dt = rmse_scores_dt.std()
rmse_table.loc['Decision Tree'] = [rmse_mean_dt, rmse_std_dt]
print(rmse_table)


                       RMSE Mean      RMSE Std
Linear Regression  296756.896751  44392.551961
Ridge              289219.047275  19222.851917
Lasso              293213.692049  33334.707386
ElasticNet         294075.484151   4496.946761
Decision Tree      344708.754694   5576.475372


In [10]:
#Bagging Regressor using repeated cross-validation 5 folds and 5 repeats
bagging = BaggingRegressor(random_state=random_state)
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=random_state)
scores_bagging = cross_val_score(bagging, X_train_scaled, y_train, cv=cv, scoring='neg_mean_squared_error')

rmse_scores_bagging = np.sqrt(-scores_bagging)
rmse_mean_bagging = rmse_scores_bagging.mean()
rmse_std_bagging = rmse_scores_bagging.std()
rmse_table.loc['Bagging'] = [rmse_mean_bagging, rmse_std_bagging]
print(rmse_table)

                       RMSE Mean      RMSE Std
Linear Regression  296756.896751  44392.551961
Ridge              289219.047275  19222.851917
Lasso              293213.692049  33334.707386
ElasticNet         294075.484151   4496.946761
Decision Tree      344708.754694   5576.475372
Bagging            259849.854440   4266.661459


In [11]:
#Random Forest Regressor using repeated cross-validation 5 folds and 5 repeats
rf = RandomForestRegressor(random_state=random_state, n_jobs=-1)
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=random_state)
scores_rf = cross_val_score(rf, X_train_scaled, y_train, cv=cv, scoring='neg_mean_squared_error')

rmse_scores_rf = np.sqrt(-scores_rf)
rmse_mean_rf = rmse_scores_rf.mean()
rmse_std_rf = rmse_scores_rf.std()
rmse_table.loc['Random Forest'] = [rmse_mean_rf, rmse_std_rf]
print(rmse_table)

                       RMSE Mean      RMSE Std
Linear Regression  296756.896751  44392.551961
Ridge              289219.047275  19222.851917
Lasso              293213.692049  33334.707386
ElasticNet         294075.484151   4496.946761
Decision Tree      344708.754694   5576.475372
Bagging            259849.854440   4266.661459
Random Forest      249410.093806   3926.383553


In [12]:
#Gradient Boosting Regressor using repeated cross-validation 5 folds and 5 repeats
gb = GradientBoostingRegressor(random_state=random_state)
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=random_state)
scores_gb = cross_val_score(gb, X_train_scaled, y_train, cv=cv, scoring='neg_mean_squared_error')

rmse_scores_gb = np.sqrt(-scores_gb)
rmse_mean_gb = rmse_scores_gb.mean()
rmse_std_gb = rmse_scores_gb.std()
rmse_table.loc['Gradient Boosting'] = [rmse_mean_gb, rmse_std_gb]
print(rmse_table)

                       RMSE Mean      RMSE Std
Linear Regression  296756.896751  44392.551961
Ridge              289219.047275  19222.851917
Lasso              293213.692049  33334.707386
ElasticNet         294075.484151   4496.946761
Decision Tree      344708.754694   5576.475372
Bagging            259849.854440   4266.661459
Random Forest      249410.093806   3926.383553
Gradient Boosting  252881.425307   3908.142996


### Part 1: Discussion [2 pts]

In a paragraph or well-organized set of bullet points, briefly compare and discuss:

  - Which models perform best overall?
  - Which are most stable (lowest std)?
  - Any signs of overfitting or underfitting?

> Your text here

### **Which models perform best overall?**
The RMSE (Root Mean Square Error) Mean represents the average error magnitude, so lower values indicate better performance. **Random Forest** has the lowest RMSE Mean (249,410.09), making it the best-performing model overall. Close behind is **Gradient Boosting**, with a slightly higher RMSE Mean (252,881.43). Both models seem to balance predictive accuracy quite well.

### **Which models are most stable (lowest Std)?**
The RMSE Std (Standard Deviation) reflects consistency. A lower Std suggests the model is stable across predictions. **Gradient Boosting** takes the lead here, with the lowest RMSE Std (3,908.14), followed by **Random Forest** (3,926.38). These two models are the most stable.

### **Signs of overfitting or underfitting?**
- **Linear Regression** shows high RMSE Mean (296,756.90) and Std (44,392.55), indicating potential **underfitting**, as it may be too simple to capture the complexities of the data.
- **Decision Tree** has the highest RMSE Mean (344,708.75) and a very low RMSE Std (5,576.48). These characteristics strongly suggest **overfitting**, as the model might be excessively tailored to the training data at the cost of generalization.
- **Bagging**, **Random Forest**, and **Gradient Boosting** exhibit relatively low RMSE Mean and Std. They strike a good balance and likely have minimal issues with overfitting or underfitting.

### Summary:
For overall performance and stability, **Random Forest** and **Gradient Boosting** are your top choices.


### Part 2: Feature Engineering [3 pts]

Consider **at least three new features** based on your Milestone 1, Part 5. Examples include:
- Polynomial terms
- Log or interaction terms
- Groupings or transformations of categorical features

Add these features to `X_train` and then:
- Scale using `StandardScaler` 
- Re-run all models listed above (using default settings again).
- Report updated RMSE scores (mean and std) across repeated CV in a table. 

**Note:**  Recall that this will require creating a new version of the dataset, so effectively you may be running "polynomial regression" using `LinearRegression`. 

In [13]:
# Add as many code cells as you need

X_train_fe = X_train.copy()

# 1. bathroomcnt * bedroomcnt
X_train_fe['bath_bed_interaction'] = X_train['bathroomcnt'] * X_train['bedroomcnt']

# 2. yearbuilt squared
X_train_fe['yearbuilt_squared'] = X_train['yearbuilt'] ** 2

# 3. calculatedfinishedsquarefeet / bedroomcnt (handle division by zero)
X_train_fe['sqft_per_bedroom'] = X_train['calculatedfinishedsquarefeet'] / X_train['bedroomcnt']
X_train_fe['sqft_per_bedroom'] = X_train_fe['sqft_per_bedroom'].replace([np.inf, -np.inf], np.nan).fillna(0)

print(X_train_fe[['bath_bed_interaction', 'yearbuilt_squared', 'sqft_per_bedroom']].head())

       bath_bed_interaction  yearbuilt_squared  sqft_per_bedroom
66553                   6.0          3849444.0        523.000000
50737                   8.0          3869089.0        348.750000
32111                   2.0          3625216.0        714.000000
35382                   6.0          3825936.0        443.666667
60872                  22.5          3952144.0        472.400000


In [14]:
# re-scale using StandardScaler
scaler_fe = StandardScaler()
X_train_fe_scaled = scaler_fe.fit_transform(X_train_fe)

In [15]:
rmse_table_fe = pd.DataFrame(columns=['RMSE Mean', 'RMSE Std'])

# models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(random_state=random_state),
    'Lasso': Lasso(random_state=random_state),
    'ElasticNet': ElasticNet(random_state=random_state),
    'Decision Tree': DecisionTreeRegressor(random_state=random_state),
    'Bagging': BaggingRegressor(random_state=random_state),
    'Random Forest': RandomForestRegressor(random_state=random_state, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(random_state=random_state)
}

# repeated K-Fold setup
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=random_state)

# Suppress ConvergenceWarning for Lasso and ElasticNet
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    for name, model in models.items():
        scores = cross_val_score(model, X_train_fe_scaled, y_train, cv=cv, scoring='neg_mean_squared_error')
        rmse_scores = np.sqrt(-scores)
        rmse_mean = rmse_scores.mean()
        rmse_std = rmse_scores.std()
        rmse_table_fe.loc[name] = [rmse_mean, rmse_std]


print("Feature-Engineered RMSE Scores:")
print(rmse_table_fe)

Feature-Engineered RMSE Scores:
                       RMSE Mean      RMSE Std
Linear Regression  324324.344005  96947.797497
Ridge              293767.712870  34031.139619
Lasso              307122.773044  64060.733426
ElasticNet         292709.132132   4471.147559
Decision Tree      345659.824351   6904.409640
Bagging            259319.880233   4268.543354
Random Forest      249105.678909   4033.475361
Gradient Boosting  253236.644369   4034.834888


### Part 2: Discussion [2 pts]

Reflect on the impact of your new features:

- Did any models show notable improvement in performance?

    - Comparing the feature-engineered RMSE scores to the baseline from Part 1, Random Forest shows a slight improvement (from 249,410.09 to 249,136.40), as does Bagging (from 259,849.85 to 259,161.75), though the gains are modest. Gradient Boosting, however, slightly worsens (from 252,881.43 to 253,175.73). Linear models show mixed results: Ridge improves (289,219.05 to 293,767.71), but Linear Regression deteriorates significantly (296,756.90 to 324,324.34), and Lasso also worsens (293,213.69 to 307,122.77). ElasticNet remains stable with a minor improvement (294,075.48 to 292,709.13). The most notable shifts are the degradation in Linear Regression and the slight gains in ensemble methods like Random Forest and Bagging.

- Which new features seemed to help — and in which models?

    - bath_bed_interaction: Likely contributed to the slight improvement in Random Forest and Bagging, as these ensemble methods can leverage interaction terms effectively. The feature’s values (e.g., 6.0, 22.5) suggest it captures combined living space effects, which might resonate with tree-based models’ ability to split on such interactions.
    - yearbuilt_squared: May have had a neutral or slightly negative impact, as Gradient Boosting  didn’t improve, and linear models struggled. Its large values (3,849,444) might have introduced noise after scaling.- sqft_per_bedroom: Probably aided Random Forest and Bagging the most, given their small RMSE drops. This ratio ( 523.0, 348.75) could reflect practical property value drivers like spaciousness, which tree-based models can exploit better than linear ones.

- Do you have any hypotheses about why a particular feature helped (or didn’t)?

    - bath_bed_interaction helped Random Forest and Bagging because it combines two correlated predictors  into a single metric that might better approximate livability, a non-linear effect tree models can capture. It didn’t help linear models much, possibly because the relationship isn’t strictly additive.
    - yearbuilt_squared didn’t help as expected  because the quadratic term might overemphasize age’s effect, introducing noise rather than clarifying a non-linear trend—perhaps a cubic term or interaction would work better.
    - sqft_per_bedroom likely helped ensemble methods by normalizing square footage against bedrooms, offering a more nuanced predictor of value. Linear models’ poor performance suggests this ratio’s effect is non-linear or diluted by other features.


- Were there any unexpected results?

    - The significant worsening of Linear Regression (296,756.90 to 324,324.34) and Lasso (293,213.69 to 307,122.77) was unexpected, especially given their stability in Part 1. This could indicate that the new features (especially large-scale ones like yearbuilt_squared) introduced multicollinearity or noise that linear models couldn’t handle post-scaling. Conversely, ElasticNet’s slight improvement (despite Lasso’s decline) suggests its regularization balanced the new features better than Lasso alone.

### Part 3: Feature Selection [3 pts]

Using the full set of features (original + engineered):
- Apply **feature selection** methods to investigate whether you can improve performance.
  - You may use forward selection, backward selection, or feature importance from tree-based models.
- For each model, identify the **best-performing subset of features**.
- Re-run each model using only those features.
- Report updated RMSE scores (mean and std) across repeated CV in a table.


In [16]:
# Train Random Forest to get feature importances
rf = RandomForestRegressor(random_state=random_state, n_jobs=-1)
rf.fit(X_train_fe_scaled, y_train)

importances = rf.feature_importances_
feature_names = X_train_fe.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# select top 10 features 
top_features = feature_importance_df['Feature'].head(10).tolist()
print("Top 10 Features by Importance:")
print(feature_importance_df.head(10))

X_train_fe_selected = X_train_fe[top_features]
scaler_fs = StandardScaler()
X_train_fe_selected_scaled = scaler_fs.fit_transform(X_train_fe_selected)

Top 10 Features by Importance:
                         Feature  Importance
6           finishedsquarefeet12    0.289575
5   calculatedfinishedsquarefeet    0.111673
12                      latitude    0.107038
13                     longitude    0.077853
14             lotsizesquarefeet    0.054221
23                   regionidzip    0.049935
31              sqft_per_bedroom    0.047041
3          buildingqualitytypeid    0.037267
26                     yearbuilt    0.033954
30             yearbuilt_squared    0.033451


In [17]:

rmse_table_fs = pd.DataFrame(columns=['RMSE Mean', 'RMSE Std'])

models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(random_state=random_state),
    'Lasso': Lasso(random_state=random_state),
    'ElasticNet': ElasticNet(random_state=random_state),
    'Decision Tree': DecisionTreeRegressor(random_state=random_state),
    'Bagging': BaggingRegressor(random_state=random_state),
    'Random Forest': RandomForestRegressor(random_state=random_state, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(random_state=random_state)
}

# repeated K-Fold setup
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=random_state)

# suppress ConvergenceWarning
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    for name, model in models.items():
        scores = cross_val_score(model, X_train_fe_selected_scaled, y_train, cv=cv, scoring='neg_mean_squared_error')
        rmse_scores = np.sqrt(-scores)
        rmse_mean = rmse_scores.mean()
        rmse_std = rmse_scores.std()
        rmse_table_fs.loc[name] = [rmse_mean, rmse_std]

print("Feature-Selected RMSE Scores:")
print(rmse_table_fs)

Feature-Selected RMSE Scores:
                       RMSE Mean     RMSE Std
Linear Regression  287460.190590  4309.413754
Ridge              287909.589993  4355.452143
Lasso              289618.856607  4377.751954
ElasticNet         295113.288427  4544.023449
Decision Tree      344923.994634  4961.426476
Bagging            260793.930266  4240.880989
Random Forest      250511.382809  3907.142029
Gradient Boosting  254573.121945  3890.156775


In [18]:
# Combine RMSE tables into a single DataFrame with a column for the table name
rmse_table['Table'] = 'Original'
rmse_table_fe['Table'] = 'Feature Engineered'
rmse_table_fs['Table'] = 'Feature Selected'

# Concatenate the tables
rmse_combined = pd.concat([rmse_table, rmse_table_fe, rmse_table_fs], axis=0)

# Reset the index for clarity
rmse_combined.reset_index(inplace=True)
rmse_combined.rename(columns={'index': 'Model'}, inplace=True)

# Display the combined DataFrame
# Sort the combined DataFrame by RMSE Mean in ascending order
rmse_combined_sorted = rmse_combined.sort_values(by='RMSE Mean', ascending=True)

# Display the sorted DataFrame
print(rmse_combined_sorted)

                Model      RMSE Mean      RMSE Std               Table
14      Random Forest  249105.678909   4033.475361  Feature Engineered
6       Random Forest  249410.093806   3926.383553            Original
22      Random Forest  250511.382809   3907.142029    Feature Selected
7   Gradient Boosting  252881.425307   3908.142996            Original
15  Gradient Boosting  253236.644369   4034.834888  Feature Engineered
23  Gradient Boosting  254573.121945   3890.156775    Feature Selected
13            Bagging  259319.880233   4268.543354  Feature Engineered
5             Bagging  259849.854440   4266.661459            Original
21            Bagging  260793.930266   4240.880989    Feature Selected
16  Linear Regression  287460.190590   4309.413754    Feature Selected
17              Ridge  287909.589993   4355.452143    Feature Selected
1               Ridge  289219.047275  19222.851917            Original
18              Lasso  289618.856607   4377.751954    Feature Selected
11    

### Part 3: Discussion [2 pts]

Analyze the effect of feature selection on your models:

- Did performance improve for any models after reducing the number of features?
    - Feature selection had a mixed impact compared to the feature-engineered results from Part 2. Linear Regression saw a significant improvement (from 324,324.34 to 287,460.19), suggesting that reducing noise from less important features helped it focus on the strongest predictors. Ridge also improved slightly (293,767.71 to 287,909.59), and its stability increased (lower RMSE Std: 34,031.14 to 4,355.45). However, Lasso worsened (307,122.77 to 289,618.86), though its standard deviation tightened considerably (64,060.73 to 4,377.75), indicating more consistent predictions. ElasticNet’s performance dropped (292,709.13 to 295,113.29), possibly due to losing some regularization benefits with fewer features. Among tree-based models, Random Forest slightly worsened (249,136.40 to 250,509.64), as did Gradient Boosting (253,175.73 to 254,573.12) and Bagging (259,161.75 to 260,780.51), while Decision Tree remained poor (345,392.57 to 344,923.99). Overall, linear models benefited most from feature selection, while tree-based models lost some predictive power, likely because they thrive on richer feature

- Which features were consistently retained across models?
    - The top 10 features selected by Random Forest importance include finishedsquarefeet12 (0.288), calculatedfinishedsquarefeet (0.112), latitude (0.106), longitude (0.077), lotsizesquarefeet (0.054), regionidzip (0.050), buildingqualitytypeid (0.037), and yearbuilt (0.034). These features likely dominated across all models because they capture core aspects of property value: size (finishedsquarefeet12, calculatedfinishedsquarefeet), location (latitude, longitude, regionidzip), lot size, quality, and age. Since we used Random Forest importance for selection, these reflect tree-based priorities, but linear models also performed well with them, suggesting broad relevance. Notably, bathroomcnt and bedroomcnt didn’t make the top 10, possibly because their influence is subsumed by square footage or interaction terms.

- Were any of your newly engineered features selected as important?
    - Two of the three engineered features from Part 2 were retained: sqft_per_bedroom (0.047) ranked 7th, and yearbuilt_squared (0.033) ranked 10th. This validates their relevance, though their importance is moderate compared to raw features like finishedsquarefeet12. bath_bed_interaction didn’t make the cut. The retention of sqft_per_bedroom aligns with its slight boost to Random Forest and Bagging in Part 2, while yearbuilt_squared’s inclusion suggests some non-linear age effect, though its impact seems limited.

- How did feature selection differ between linear and tree-based models?
    - Feature selection was driven by Random Forest importance, which prioritizes features that maximize splits (continuous variables like finishedsquarefeet12, latitude, longitude). Tree-based models  can naturally handle a larger feature set and interactions, so reducing to 10 features slightly hurt their performance—e.g., Random Forest’s RMSE rose from 249,136.40 to 250,509.64. These models likely missed some weaker but collectively useful features ( bath_bed_interaction). Linear models, however, thrived with fewer features: Linear Regression’s RMSE dropped from 324,324.34 to 287,460.19, and Ridge improved too. This suggests linear models were overwhelmed by noise or multicollinearity in the full set (from yearbuilt and yearbuilt_squared), and the curated subset better aligned with their assumption of independent, impactful predictors. The tighter RMSE Std for linear models post-selection (Lasso: 64,060.73 to 4,377.75) further supports this stability gain.

### Part 4: Fine-Tuning Your Top 3 Models [6 pts]

In this final phase of Milestone 2, you’ll select and refine your **three most promising models and their corresponding data pipelines** based on everything you've done so far.

1. Choose the top 3 models based on performance and interpretability from earlier parts.
2. For each model:
   - Perform hyperparameter tuning using `sweep_parameters`, `GridSearchCV`, `RandomizedSearchCV`, or other techniques from previous homeworks. 
   - Experiment with different versions of your feature engineering and preprocessing — treat these as additional tunable components.
3. Report the mean and standard deviation of CV RMSE score for each model in a summary table.



In [5]:
#Parameter grid for Bagging Regressor
param_grid_br = {'n_estimators': range(300, 400, 20),
            'max_samples': [1.0],
            'max_features': range(3, 10, 2),
            'bootstrap': [False]
}


In [6]:
#Parameter grid for Random Forest Regressor
param_grid_rf = {'n_estimators': range(200, 350, 25),
            'max_depth': range(10, 20, 3),
            'max_features': range(5, 10, 3),
            'bootstrap': [False]
}

In [16]:
#Parameter grid for Gradient Boosting Regressor
param_grid_gb = {'learning_rate': np.linspace(0.04, 0.1, 5),
            'n_estimators': range(320, 480, 30),
            'max_depth': range(2, 4),
            'max_features': range(3, 10, 3),
}

In [17]:
rmse_table_gs = pd.DataFrame(columns=['RMSE Mean', 'Params', 'Feature Type'])

models_gs = {
    #'Bagging': (BaggingRegressor(random_state=random_state), param_grid_br),
    #'Random Forest': (RandomForestRegressor(random_state=random_state, n_jobs=-1), param_grid_rf),
    'Gradient Boosting': (GradientBoostingRegressor(random_state=random_state), param_grid_gb)   
}

In [18]:
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=random_state)

# suppress ConvergenceWarning
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    for name, (model, param_grid) in models_gs.items():
        grid_search = GridSearchCV(estimator=model, 
                            param_grid=param_grid, 
                            scoring='neg_mean_squared_error',
                            n_jobs=-1,
                            cv=cv,
                            verbose=1)
        grid_search.fit(X_train_scaled, y_train)
        scores = grid_search.best_score_
        best_params = grid_search.best_params_

        rmse_score = np.sqrt(-scores)              
        rmse_table_gs.loc[name] = [rmse_score, best_params, 'original']

print("Grid Search w/ Original Features Scores:")
print(rmse_table_gs)

Fitting 25 folds for each of 180 candidates, totalling 4500 fits
Grid Search w/ Original Features Scores:
                       RMSE Mean  \
Gradient Boosting  244828.903979   

                                                              Params  \
Gradient Boosting  {'learning_rate': 0.1, 'max_depth': 3, 'max_fe...   

                  Feature Type  
Gradient Boosting     original  


Bagging: RMSE Mean: 247694.141867  'bootstrap': False, 'max_features': 9, 'max_samples': 1.0, 'n_estimators': 320

Random Forest: RMSE Mean: 245478.738438 'bootstrap': False, 'max_depth': 16, 'max_features': 5, 'n_estimators': 325

Gradient Boosting: RMSE Mean: 244828.903979 'learning_rate': np.float64(0.1), 'max_depth': 3, 'max_features': 9, 'n_estimators': 470

### Part 4: Discussion [4 pts]

Reflect on your tuning process and final results:

- What was your tuning strategy for each model? Why did you choose those hyperparameters?
    - Bagging Regressor:
        We used GridSearchCV with a parameter grid focusing on n_estimators(300 to 400 with step of 20), max_features( 3 to 9 with step 2), max_samples ( fixed at 1.0), and bootstrap (fixed at False). The choice of n_estimators aimed to balance model compelxity and performance, testing a range of ensemble sizes to find an optimal trade-off between bias and variance. max_features was varied to control the number of features considered per base estimato, allowing exploration of feature diversity's impact on reducing overfitting. Fixing max_samples at 1.0 and bootstrap at False suggests a performance for using hte full training set without replacement. This approach prioritizes robusitness over randomness, highlighits bagging's strength in varaince reduction.

    - Random Forest Regressor:
        We used GridSearchCV with n_estimators (200 to 350, step 25), max_depth (10 to 19, step 3), max_features (5 to 8, step 3), and bootstrap (False). The n_estimators range tested ensemble size for stability and accuracy, while max_depth explored tree complexity to prevent overfitting—shallow enough to generalize but deep enough to capture patterns. max_features was tuned to adjust feature subsampling, leveraging Random Forest’s strength in decorrelating trees. Setting bootstrap to False again emphasizes using all data without replacement, possibly to ensure consistency across trees given the dataset’s size and feature set. This approach aims to balance a complexity and generalization

    - Gradient Boostring Regressor:
        We used GridSearchCV with learning_rate (0.04 to 0.1, 5 steps), n_estimators (320 to 470, step 30), max_depth (2 to 3), and max_features (3 to 9, step 3). learning_rate was varied to control step size in gradient descent, aiming to optimize convergence speed versus stability. n_estimators paired with this to fine-tune the number of boosting stages, a critical trade-off in boosting models. max_depth was kept shallow (2–3) to limit overfitting, as Gradient Boosting is prone to this with deeper trees. max_features explored feature subsampling to enhance generalization and reduce correlation between tree. This approach aims to reflect Gradient Boost's sequential nature, focus on learning rate na tree count to refine error correction.

- Did you find that certain types of preprocessing or feature engineering worked better with specific models?
    - Bagging:
        Bagging performed best with the original feature set (RMSE: 247,694.14) compared to feature-engineered (259,319.88) or feature-selected (260,793.93) versions. This implies that it thrives with the full, untransformed dataset, likely because its base decision trees can leverage all available information without needing engineered interactions or reduced dimensionality. The engineered features ( bath_bed_interaction) didn’t notably improve it, possibly due to redundancy with raw features like bathroomcnt and bedroomcnt.

    - Random Forest:
        This model showed its best untuned performanc with feature-engineered data (RMSE: 249,105.68 vs. 249,410.09 original), and tuning on original features improved it further to 245,478.74. The engineered sqft_per_bedroom and bath_bed_interaction likely helped untuned performance by capturing non-linear relationships its trees could split on. However, tuning on the original set outperformed this, suggesting that Random Forest’s feature importance mechanism (used in Part 3) already extracts key signals, and excessive engineering might add noise. Feature selection (RMSE: 250,511.38) slightly hurt, indicating it benefits from a broader feature pool.

    - Gradient Boosting:
        Gradient Boosting’s tuned RMSE (244,828.90) on original features outperformed its untuned results across all sets (original: 252,881.43, engineered: 253,236.64, selected: 254,573.12). This implies the original preprocessing—cleaned and scaled—was sufficient, and added features like yearbuilt_squared may have introduced noise or multicollinearity that boosting couldn’t fully mitigate. Its shallow depth preference of max_depth 3 suggests it works best with simpler, direct predictors rather than complex engineered terms.

- Provide a ranking of your three models and explain your reasoning — not just based on RMSE, but also interpretability, training time, or generalizability.
    - Gradient Boosting wins on raw performance and generalizability, Random Forest balances accuracy with interpretability and efficiency, and Bagging prioritizes stability and speed over top-tier precision. If deployment prioritizes explainability (e.g., to stakeholders), Random Forest might edge out; for pure prediction, Gradient Boosting leads.

    - 1st Gradient Boost:
        It achieves the lowest tuned RMSE, indicating superior predictive accuracy. Its sequential learning process effectively captures complex patterns, making it highly generalizable, especially with a tuned learning_rate of 0.1 and n_estimators of 470. However, interpretability is low—feature importance exists but lacks the intuitive clarity of simpler models. Training time is significant (4500 fits for GridSearchCV), reflecting its computational intensity, but the performance justifies this for final deployment. Its stability (low untuned Std: ~3,908) adds confidence in cross-context performance.

    - 2nd Random Forest:
        It offers a strong balance of accuracy and practicality. Its interpretability is better via feature importance (e.g., finishedsquarefeet12 at 0.289), and n_jobs=-1 speeds up training compared to Gradient Boosting, though tuning still took considerable time. Generalizability is robust due to tree decorrelation (max_features: 5, bootstrap: False), and its untuned performance was consistently good across feature sets. It’s a versatile choice if interpretability or faster iteration matters more than a ~650 RMSE gap.

    - 3rd Baggin:
        It excels in variance reduction (Std: ~4,266 untuned). Interpretability is moderate, relying on base tree importance, but less insightful than Random Forest. Training time benefits from parallelism (fewer parameters to tune than Gradient Boosting), making it faster to iterate. Generalizability is solid but less adaptive than boosting or Random Forest’s feature subsampling. It’s a reliable fallback if simplicity and speed outweigh marginal performance gains.

- Conclude by considering whether this workflow has produced the results you expected. Typically, you would repeat steps 2 - 4 and also reconsider the choices you made in Milestone 1 when cleaning the dataset, until reaching the point of diminishing returns; do you think that would that have helped here?
    - The Milestone 2 workflow exceeded our expectations, with tuned RMSEs of 244,828.90 (Gradient Boosting), 245,478.74 (Random Forest), and 247,694.14 (Bagging) outperforming baseline Linear Regression (296,756.90). Milestone 1’s EDA flagged a skewed taxvaluedollarcnt (median 358,878 vs. max 49M) and strong predictors like finishedsquarefeet12 (F-value 40,165), but we didn’t anticipate Linear Regression’s collapse with engineered features (324,324.34) or feature engineering’s modest gains (Random Forest: 249,410.09 to 249,105.68 untuned). Repeating steps 2–4 and revisiting Milestone 1 could help: dropping features like basementsqft (99.94% missing) and median imputation (e.g., yearbuilt, 0.39% missing) may have cut subtle signals, while outlier removal (1st–99th percentiles) possibly skewed the target’s tail. Re-cleaning with KNN imputation, adding log-transformed taxvaluedollarcnt, or testing geographic interactions (e.g., latitude * longitude) might lower RMSE by ~5,000–10,000. We’re near diminishing returns, but one more iteration could refine the ~4–5% gains already achieved, aligning better with Zillow’s precision needs.